# Day 1

#Numpy

In [1]:
#Exercise 1 answer
def p(x,coeff):
    a = np.array(coeff)
    for i  in range(len(a)):
        p[i] = a[i]*(x**i)
    f = np.sum(p)
    return f

In [ ]:
#Exercise 2
"""
we want to use the inverse cdf method to draw values from a discrete distribution
"""

import numpy as np
import scipy as sc

"""
we want to write our method such that for any given probability vector q we can obtain 
k draws from the discrete distribution corresponding to q
"""

def invcdf(q,k):
    prob = np.array(q)
    p = np.cumsum(prob)
    x = []
    for i in range(k):
        u = np.random.uniform()
        a = p.searchsorted(u)
        x.append(a)
    return x
    

#Scipy

In [ ]:
#Exercise 1 Answer

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

"""
plotting the profit function
"""
p=5
q = np.linspace(0,2,1000)
pi = p*q - np.exp(q) - 0.5*(q**2)

plt.figure
plt.plot(q,pi)
plt.xlabel('q', fontsize=20)
plt.ylabel('$\pi$(q)', fontsize=20)
plt.title(f'Profit Function, price = {p}', fontsize=20)
plt.show()

import scipy as sc
from scipy import optimize

def profitmax(p,x):
    answer = optimize.fminbound(lambda q: -p*q + np.exp(q) + 0.5*(q**2), 0, x)
    return answer 

def profit(p,q):
    pi = p*q - np.exp(q) - 0.5*(q**2)
    return pi

q = profitmax(5,5)
print(f'The maximised level of profit is {profit(5,q)}')
print(f'Quantity of output at profit maximum is {q}')

#Numba

In [ ]:
#Exercise 1
import numpy as np
from numba import jit

@jit
def estimate_pi(n):

    U = np.random.uniform(-1,1,n)
    V = np.random.uniform(-1,1,n)
    count = 0
    for i in range(n):
        d = np.sqrt(U[i]**2 + V[i]**2)
        if d <= 1:
            count += 1
    pi = 4*count/n
    return pi

estimate_pi(1000000)

In [ ]:
#Exercise 2

import numpy as np

"""
denote the low state at 0 and the high state as 1
"""
state = 1
x = []
n=1000000

for i in range(n):
    u = np.random.uniform()
    x.append(state)
    if state == 1:
        if u<0.2:      #probability of moving from high to low is 0.2
            state = 0
        else:
            state = 1
    else:
        if u<0.1:     #probability of moving from low to high is 0.1
            state = 1
        else:
            state = 0
    
timeH = sum(x)/n
print(timeH)

# Day 2

#Writing Good Code

In [ ]:
#Exercise 1
#Improving the code

import numpy as np
from scipy import optimize
import scipy as sc
import matplotlib.pyplot as plt

%matplotlib inline

#parameters
α = 0.1
β = 1
γ = 1
δ = 1
lb, ub = 2,4

#defining the excess demand
def excess_demand(p):
    h = γ*(p**(-δ)) - np.exp(α*p) + β
    return h

#solving numerically for the equilibrium price and quantity

price_eq = sc.optimize.brentq(excess_demand, lb,ub)
quantity_eq = γ*(price_eq**(-δ))

grid = np.linspace(2, 4, 100)

print(f'Equilibrium price is {price_eq: .2f}')
print(f'Equilibrium quantity is {quantity_eq: .2f}')

#plotting the demand and supply curves
price = np.linspace(lb,ub,100)
qd = γ*(price**(-δ))
qs = np.exp(α*price) - β

fig, ax = plt.subplots()
ax.plot(qd, grid, 'b-', lw=2, label='Demand')
ax.plot(qs, grid, 'g-', lw=2, label='Supply')
fig.set_size_inches(10.5, 7.5)

ax.set_xlabel('Quantity',fontsize=20)
ax.set_ylabel('Price',fontsize=20)
ax.legend(loc='upper right')
ax.set_title('Market Diagram',fontsize=20)
plt.axhline(y=price_eq, color='r', linestyle='--')
plt.axvline(x=quantity_eq, color='r', linestyle='--')

plt.show()

#Parallelisation

In [ ]:
#Exercise 1

"""
To estimate π I draw a circle of radius one inside a square with side length 2.  The ratio of areas of the circle
to the square is π/4.  Therefore, by 'throwing' dots at the 2x2 grid centred at 0, we can estimate π by the
proportion of dots that lie inside the circle.
"""

import numpy as np
from numba import prange

n = 1_000_000
count = 0
for i in prange(n):
    u,v = np.random.uniform(-1,1),np.random.uniform(-1,1)
    d = np.sqrt(u**2 + v**2)
    if d<=1:
        count += 1
        
pi = 4*count/n

print(f'The estimated value of π is {pi}')


#AR1 Processes

In [1]:
#Exercise 1

import numpy as np

mu = 3
sigma = 0.6

m = 10_000_000

moment = 10

def doublefactorial(n):
     if n <= 0:
         return 1
     else:
         return n * doublefactorial(n-2)

X = mu + sigma*np.random.randn(m)
Xbar = sum(X)/len(X)
k = np.arange(1,moment+1,1)
M = np.empty(len(k))
error = np.empty(len(k))

for i in range(len(k)):
    if k[i]%2 != 0:
        k[i] = 0
    M[i] = (sigma**k[i])*doublefactorial(k[i]-1)
    error[i] = sum((X-Xbar)**k[i])/len(X) - M[i]
    print(f'The error in approximation for the {i+1}th moment, {M[i]}, is {error[i]}')

KeyboardInterrupt: 

In [ ]:
#Exercise 2

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline


def kerneldensity(a,b,n=500):
    X = sc.stats.beta.rvs(a, b, size=n)
    grid_size = 1000
    x = np.linspace(min(X)-np.std(X),max(X)+np.std(X),grid_size)

    #1.06 rule of thumb
    h = ((4/3)**1.5)*(n**-0.2)*np.std(X)

    non_parametric = np.empty(grid_size)

    for i, numbers in enumerate(x):
        n_p = (1/(n*h))*sum(sc.stats.norm.pdf((X-numbers)/h))
        non_parametric[i] = n_p

    fig, ax = plt.subplots()
    ax.plot(x,non_parametric)
    ax.plot(x,sc.stats.beta.pdf(x,a,b))
    ax.set_xlabel('x')
    ax.set_ylabel('KDE(x),f(x)')
    ax.set_title(f'Kerndel Density Estimate of Beta Distribution with Parameters {a,b}')
    plt.show()

α = [2,5,0.5]
β = [2,5,0.5]

for i in range(len(α)):
    kerneldensity(α[i],β[i])

In [ ]:
#Exercise 3

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

n=2000

a = 0.9
b= 0
c = 0.1
mu = -3
s = 0.2

W = np.random.randn(n)

X = mu + s*np.random.randn(n)
Y = a*mu+b + np.sqrt((a**2)*(s**2)+(c**2))*np.random.randn(n)

X_t = a*X + b + c*W

#kernel density estimation
grid_size = n
x = np.linspace(min(X_t)-np.std(X_t),max(X_t)+np.std(X_t),grid_size)

#1.06 rule of thumb
h = ((4/3)**1.5)*(n**-0.2)*np.std(X_t)

non_parametric = np.empty(grid_size)

for i, numbers in enumerate(x):
    n_p = (1/(n*h))*sum(sc.stats.norm.pdf((X_t-numbers)/h))
    non_parametric[i] = n_p
    


fig, ax = plt.subplots()
ax.plot(x,non_parametric)
ax.plot(x,sc.stats.norm.pdf(x,a*mu+b,(a**2)*(s**2)+(c**2)))
ax.set_xlabel('x')
ax.set_ylabel('KDE(x),f(x)')
ax.set_title(f'Kerndel Density Estimate of One Step Ahead of AR(1) with N({a*mu+b,(a**2)*(s**2)+(c**2)})')
ax.legend(['Kernel Density Estimate','True Distribution'])
plt.show()
    


#Heavy Tails

In [ ]:
#Exercise 1
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(11)

n = 120

norm_draws_s2 = 2*np.random.randn(n)
norm_draws_s12 = 12*np.random.randn(n)
cauchy_draws = np.random.standard_cauchy(size=n)

fig, ax = plt.subplots()

plt.subplot(3,1,1)
plt.scatter(range(n),norm_draws_s2)

plt.subplot(3,1,2)
plt.scatter(range(n),norm_draws_s12)

plt.subplot(3,1,3)
plt.scatter(range(n),cauchy_draws)

In [ ]:
#Exercise 3
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(11)

α = [1.15,1.5,1.75]
n=120

def plotter(a):
    X = np.random.pareto(a,n)
    fig,ax = plt.subplots()
    ax.plot(X)
    
for a in α:
    plotter(a)

#Scalar Dynamics

In [ ]:
#Exercise 1

"""
For the difference equation x_{t+1} = ax_t + b study the dynamics over time series length 
T for a variety of a \in (-1,1), b=1
"""
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

b=1
a_pos = np.arange(0.1,1,0.1)
a_neg = np.arange(-0.9,0,0.1)

T = 20

x = np.random.randn()
y = np.random.randn()
X = np.empty((int(len(a_pos)),T))
Y = np.empty((int(len(a_neg)),T))

for a in range(len(a_pos)):
    for t in range(T):
        x = a_pos[a]*x+b
        y = a_neg[a]*y+b
        X[a,t] = x
        Y[a,t] = y
        
fig

fig = plt.figure()

plt.subplot(2, 1, 1)
fig.set_size_inches(18.5, 10.5)
for a in range(len(a_pos)):
    plt.plot(range(T),X[a,:])

plt.subplot(2, 1, 2)
fig.set_size_inches(18.5, 10.5)
for a in range(len(a_neg)):
    plt.plot(range(T),Y[a,:])

plt.show()


# Day 3

#Kesten Processes

In [ ]:
#Exercise 1

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

def garch(time, σ_0 = 0, α_0 = 0.00001, α_1 = 0.1, β = 0.9):
    stock_returns = []
    sigma = np.empty(time)
    sigma[0] = σ_0
    for t in range(time-1):
        ε = np.random.randn()
        sigma[t+1] = α_0 + sigma[t]*(α_1*ε + β)
        ϵ = np.random.randn()
        r = np.sqrt(sigma[t+1])*ϵ
        stock_returns.append(r)
    return stock_returns
        
time = 15*250
returns = garch(time)


###---------------------Comparison to Nasdaq Composite Index
import yfinance as yf
import pandas as pd

s = yf.download('^IXIC', '2006-1-1', '2019-11-1')['Adj Close']

r = s.pct_change()


fig, axes = plt.subplots(2, 1, figsize=(11, 6))

ax = axes[0]
ax.plot(returns,alpha=0.7)
ax.set_ylabel('returns', fontsize=12)
ax.set_xlabel('date', fontsize=12)

axes[1].plot(r,alpha=0.7)
axes[1].set_ylabel('returns', fontsize=12)
axes[1].set_xlabel('date', fontsize=12)

plt.show()

In [ ]:
#Exercise 4

μ_a = -0.5        # location parameter for a
σ_a = 0.1         # scale parameter for a
μ_b = 0.0         # location parameter for b
σ_b = 0.5         # scale parameter for b
μ_e = 0.0         # location parameter for e
σ_e = 0.5         # scale parameter for e
s_bar = 1.0       # threshold

s_init = 1.0      # initial condition for each firm

import numpy as np
from numba import prange, jit
import matplotlib.pyplot as plt
%matplotlib inline


@jit
def firm_dynamics(T,M):
    firm = np.empty((M,T))
    firm[:,0] = s_init
    for m in prange(M):
        for t in range(T-1):
            if firm[m,t] < s_bar:
                e = np.exp(μ_e + σ_e*np.random.randn())
                firm[m,t+1] = e
            else:
                a = np.exp(μ_a + σ_a*np.random.randn())
                b = np.exp(μ_b + σ_b*np.random.randn())
                firm[m,t+1] = a*firm[m,t] + b
    return firm

T = 500           # sampling date
M = 10_000_000     # number of firms
firm = firm_dynamics(T,M)

size = 1000   #size of the size rank plot

log_size = np.log(sorted(firm[0:size,-1],reverse = True))
log_rank = np.log(range(1,size+1))

fig, ax = plt.subplots()
ax.plot(log_size,log_rank)
ax.set(xlabel='log size', ylabel='log rank')

plt.show()

# Day 4

#Short Path

In [ ]:
#Exercise 1

num_nodes = 100
destination_node = 99

def map_graph_to_distance_matrix(in_file):

   # First let's set of the distance matrix Q with inf everywhere
   Q = np.ones((num_nodes, num_nodes))
   Q = Q * np.inf

   # Now we read in the data and modify Q
   infile = open(in_file)
   for line in infile:
      elements = line.split(',')
      node = elements.pop(0)
      node = int(node[4:])    # convert node description to integer
      if node != destination_node:
          for element in elements:
              destination, cost = element.split()
              destination = int(destination[4:])
              Q[node, destination] = float(cost)
   Q[destination_node, destination_node] = 0

   infile.close()
   return Q

Q = map_graph_to_distance_matrix('graph.txt')
print(Q)

num_nodes = len(Q)
J = np.zeros(num_nodes, dtype=np.float64)       # Initial guess
next_J = np.empty(num_nodes, dtype=np.float64)  # Stores updated guess
max_iter = 500
i = 0

while i < max_iter:
   for v in range(num_nodes):
      next_J[v] = np.min(Q[v, :] + J)
   if np.equal(next_J, J).all():
      break
   else:
      J[:] = next_J   # Copy contents of next_J to J
      i += 1

print("The cost-to-go function is", J)

path = [0]
number = 0
while number != 99:
    k = Q[number,:] + J
    number = k.argmin()
    path.append(number)

print(path)

#McCall Model

In [ ]:
#Exercise 1

import numpy as np
from numba import jit, jitclass, float64
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from quantecon.distributions import BetaBinomial

mcm = McCallModel()

@jit
def invcdf(q):
    prob = np.array(q)
    p = np.cumsum(prob)
    u = np.random.uniform()
    a = p.searchsorted(u)
    return a

from numba import jit, njit, prange

c_vals = np.linspace(10,40,25)

wages_c = []

for C_value in c_vals:
    
    mcm.c = C_value
    wage = compute_reservation_wage_two(mcm)
    wages_c.append(wage)

print(wages_c)

number_agents = 20000 
time = np.empty((20000,len(wages_c)))

n,a,b = 50, 200, 100
x = []
for i in range(50):
    e = BetaBinomial(n,a,b).pdf()[i]
    x.append(e)
    
@jit
def unemployment(reserve_wage,number_agents=20000):
    agents = np.zeros(number_agents)
    
    for i in prange(len(agents)):
        w = invcdf(x) + 10
        while w < reserve_wage:
            w = invcdf(x) + 10
            agents[i] += 1
            
    return agents

average_length = []
for reserve_wage in wages_c:
    unemployment_length = np.mean(unemployment(reserve_wage))
    average_length.append(unemployment_length)
    print(f'The average unemployment length for reservation wage {reserve_wage} is {unemployment_length} days')
    
plt.plot(c_vals,average_length)
plt.xlabel('c values')
plt.ylabel('average length of unemployment')
plt.show()

In [ ]:
#Exercise 2

#Exercise 2

import numpy as np
from numba import jit, jitclass, float64
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from quantecon.distributions import BetaBinomial




mccall_data = [
    ('c', float64),      # unemployment compensation
    ('β', float64),      # discount factor
    ('w', float64[:]),   # array of wage values, w[i] = wage at state i
    ('q', float64[:])    # array of probabilities
]

@jitclass(mccall_data)
class McCallModel:

    def __init__(self, c=25, β=0.99, w=w_default, q=q_default):

        self.c, self.β = c, β
        self.w, self.q = w_default, q_default

    def bellman(self, i, v):
        """
        The r.h.s. of the Bellman equation at state i.
        """
        # Simplify names
        c, β, w, q = self.c, self.β, self.w, self.q
        # Evaluate right hand side of Bellman equation
        X = np.exp(μ + σ*np.random.randn(S))
        
        max_value = max(w[i] / (1 - β), c + β * np.sum(v * q))
        return(max_value)
    
mcm = McCallModel()

def compute_reservation_wage_two(mcm, max_iter=500, tol=1e-5, σ = 0.5, μ = 2.5):  #tol = tolerance

    # Simplify names
    c, β, w, q = mcm.c, mcm.β, mcm.w, mcm.q

    # == First compute h == #

    h = np.sum(w * q) / (1 - β)
    i = 0
    error = tol + 1
    
    S = 10_000
    w = np.exp(μ + σ*np.random.randn(S))
    simul = []
    while i < max_iter and error > tol:
        for wage in w:
            s = np.maximum(wage/(1-β),h)
            simul.append(s)
        h_next = c + β * np.mean(simul)
        error = np.abs(h_next - h)
        i += 1

        h = h_next

    # == Now compute the reservation wage == #

    return (1 - β) * h

compute_reservation_wage_two(mcm)